In [1]:
%pylab inline

ModuleNotFoundError: No module named 'matplotlib'

In [2]:
num_steps=7000 # We know (empirically) 7000 steps to go from minimum to maximum

x=np.arange(num_steps)

print(3000000/num_steps) # Average inter-step interval

NameError: name 'np' is not defined

In [ ]:
desired_y=(49.4145 / 7000 * (x+1.0) ) # 49.4145: Total desired vertical mvmt (?)

In [ ]:
desired_y[0:5],desired_y[-5:]

In [ ]:
# The measured movement follows this quadratic behavior. (Empirical, fit to a quadratic)
# So, invert to find the time between steps (the xs) that yield equally spaced desired_y's
a=3.8687
b=4.8654
c=0.4721
desired_times=(-b+np.sqrt(b**2-4*a*(-desired_y)))/(2*a)
intervals=np.diff(desired_times)

In [ ]:
# Time between steps. This gets shorter as time goes on, since the motor needs
# to "speed up" to compensave for the nonlinear behavior of the vertical movement
# of the scissors lift (compared to the linear horizontal motion).
plt.plot(x[1:],intervals)

In [ ]:
intervals[0:],intervals[-5:] # These match Chloe's spreadsheet

In [ ]:
intervals_usec = intervals * 1000000 # Convert to usec

In [ ]:
#intervals_usec = intervals_usec * 0 + 250 # To test a constants inverval

In [ ]:
# Scale the intervals to uint8 for the table, going from 0-254 (linear)
# We lose precision, of course, but hopefully with rounding are
# still close to the desired behavior.

INTERVAL_TABLE_MAX=254.0 # uint8, from 0-254

expander_exponent=11 # To get a more accurate int, multiply/divide by this (2k)

interval_min=np.round( np.min(intervals_usec) )
interval_max=np.max(intervals_usec)
scaler=((interval_max-interval_min)/INTERVAL_TABLE_MAX)
scaler_expanded=np.round( scaler*(2**expander_exponent) )
if scaler==0: scaler=1 # For constants scalar might be 0

intervals_int8=np.round( (intervals_usec-interval_min)/scaler )
intervals_int8_recon = np.array(
    intervals_int8 * scaler_expanded/(2**expander_exponent) + interval_min, dtype=int)

In [ ]:
interval_min,interval_max,scaler,scaler_expanded,intervals_int8

In [ ]:
np.sum(intervals_int8_recon) # Should be close to 3sec

In [ ]:
intervals_int8[0:5],intervals_int8[-5:]

In [ ]:
fil=open('lookup_table1.h','wt')
fil.writelines('const unsigned int table1_info[] = {\n')
fil.writelines('\t%d, //num_steps\n'%num_steps);
fil.writelines('\t%d, //const unsigned int table_scaler = ;\n'%scaler_expanded);
fil.writelines('\t%d, //const unsigned int table_expander_exponent = ;\n'%expander_exponent);
fil.writelines('\t%d}; //const unsigned int table_interval_min = };\n'%interval_min);
fil.writelines('const PROGMEM uint8_t table1[] = {\n');
for nelement in intervals_int8:
    fil.writelines('%d,\n'%nelement)
fil.writelines('};\n');
fil.close()